In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer

제시된 다산콜센터 데이터에서 “문서” 컬럼을 독립변수로 하여 “분류” 컬럼 을 예측하는 딥러닝 모델을 개발하고

모델 개발 과정과 테스트 데이터셋에 대한 accuracy를 답안으로 작성하시오

### 데이터 확인

In [2]:
df = pd.read_csv("./data/다산콜재단.csv")

In [3]:
df.head()

,번호,분류,제목,내용,내용번호,문서
0,2645,복지,아빠 육아휴직 장려금,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,23522464,아빠 육아휴직 장려금아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려...
1,2644,경제,[서울산업진흥원] 서울메이드란?,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...,23194045,[서울산업진흥원] 서울메이드란?서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하...
2,2642,복지,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",광진맘택시 운영임산부영아 양육가정 전용 택시 업무개요 교통약자인 임산부와 영아가정...,22904492,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)광진맘택시 운영임산부영아 양육가정 ..."
3,2641,복지,마포 뇌병변장애인 비전센터,마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학...,22477798,마포 뇌병변장애인 비전센터마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 ...
4,2640,행정,2021년도 중1·고1 신입생 입학준비금 지원,년도 중고 신입생 입학준비금 지원 업무개요 서울시는 전국 최초로 년도부터 개 자...,22227896,2021년도 중1·고1 신입생 입학준비금 지원년도 중고 신입생 입학준비금 지원 업...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2138 entries, 0 to 2137
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   번호      2138 non-null   int64 
 1   분류      2138 non-null   object
 2   제목      2138 non-null   object
 3   내용      2138 non-null   object
 4   내용번호    2138 non-null   int64 
 5   문서      2138 non-null   object
dtypes: int64(2), object(4)
memory usage: 100.3+ KB


In [5]:
df["분류"].value_counts()

분류
행정    1098
경제     823
복지     217
Name: count, dtype: int64

In [6]:
df.describe()

,번호,내용번호
count,2138.000000,2.138000e+03
mean,1414.881197,3.224558e+06
std,688.608670,2.154023e+06
min,3.000000,2.894089e+06
25%,863.250000,2.895404e+06
50%,1424.500000,2.896546e+06
75%,1989.750000,2.897575e+06
max,2645.000000,2.352246e+07


In [7]:
df.shape

(2138, 6)

### 간단한 전처리

In [8]:
# 한글 외의 다른 문자 제거
df["문서"] = df["문서"].map(lambda x: re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", x))

In [9]:
df.head()

,번호,분류,제목,내용,내용번호,문서
0,2645,복지,아빠 육아휴직 장려금,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,23522464,아빠 육아휴직 장려금아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려...
1,2644,경제,[서울산업진흥원] 서울메이드란?,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...,23194045,서울산업진흥원 서울메이드란서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새...
2,2642,복지,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",광진맘택시 운영임산부영아 양육가정 전용 택시 업무개요 교통약자인 임산부와 영아가정...,22904492,광진맘택시 운영임산부영아 양육가정 전용 택시광진맘택시 운영임산부영아 양육가정 전용 ...
3,2641,복지,마포 뇌병변장애인 비전센터,마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학...,22477798,마포 뇌병변장애인 비전센터마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 ...
4,2640,행정,2021년도 중1·고1 신입생 입학준비금 지원,년도 중고 신입생 입학준비금 지원 업무개요 서울시는 전국 최초로 년도부터 개 자...,22227896,년도 중고 신입생 입학준비금 지원년도 중고 신입생 입학준비금 지원 업무개요 서울...


In [10]:
# 10글자 미만의 문서가 없는 것을 확인
len(df[df["문서"].map(lambda x: len(x.strip()) < 10)])

0

In [11]:
df[df["문서"].map(lambda x: len(x.strip()) < 10)].head()

,번호,분류,제목,내용,내용번호,문서


In [12]:
# 중복 데이터 확인
df[df["문서"].duplicated(keep = False)].sort_values("문서")

,번호,분류,제목,내용,내용번호,문서
649,1874,행정,불문경고도 소청심사의 대상이 되나요?,불문경고도 소청심사의 대상이 되나요불문경고는 신분상의 불이익을 초래하는 법률상의 효...,2895139,불문경고도 소청심사의 대상이 되나요불문경고도 소청심사의 대상이 되나요불문경고는 신분...
650,1873,행정,불문경고도 소청심사의 대상이 되나요?,불문경고도 소청심사의 대상이 되나요불문경고는 신분상의 불이익을 초래하는 법률상의 효...,2895150,불문경고도 소청심사의 대상이 되나요불문경고도 소청심사의 대상이 되나요불문경고는 신분...


In [13]:
# 중복 제거
df = df.drop_duplicates(subset = ["문서"])

In [14]:
df.shape

(2137, 6)

### 불필요한 피처 제거

- 독립변수 '문서'와 종속변수 '분류' 를 제외한 모든 피처 제거

In [15]:
df = df.drop(["번호", "제목", "내용", "내용번호"], axis = 1)

In [16]:
df.head()

,분류,문서
0,복지,아빠 육아휴직 장려금아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려...
1,경제,서울산업진흥원 서울메이드란서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새...
2,복지,광진맘택시 운영임산부영아 양육가정 전용 택시광진맘택시 운영임산부영아 양육가정 전용 ...
3,복지,마포 뇌병변장애인 비전센터마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 ...
4,행정,년도 중고 신입생 입학준비금 지원년도 중고 신입생 입학준비금 지원 업무개요 서울...


### 토큰화

In [17]:
okt = Okt()

In [18]:
df["token"] = df["문서"].map(lambda x: okt.morphs(x, stem = True))

In [19]:
df.head()

,분류,문서,token
0,복지,아빠 육아휴직 장려금아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려...,"[아빠, 육아휴직, 장려, 금, 아빠, 육아휴직, 장려, 금, 업무, 개요, 남성,..."
1,경제,서울산업진흥원 서울메이드란서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새...,"[서, 울, 산업, 진흥, 원, 서, 울, 메이드, 란, 서울, 산업, 진흥, 원,..."
2,복지,광진맘택시 운영임산부영아 양육가정 전용 택시광진맘택시 운영임산부영아 양육가정 전용 ...,"[광진, 맘, 택시, 운영, 임산부, 영아, 양육, 가정, 전용, 택시, 광진, 맘..."
3,복지,마포 뇌병변장애인 비전센터마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 ...,"[마포, 뇌, 병변, 장애인, 비다, 센터, 마포, 뇌, 병변, 장애인, 비다, 센..."
4,행정,년도 중고 신입생 입학준비금 지원년도 중고 신입생 입학준비금 지원 업무개요 서울...,"[년도, 중고, 신입생, 입학, 준비, 금, 지원, 년도, 중고, 신입생, 입학, ..."


In [20]:
df.to_csv("./다산콜재단_token.csv")

### 정수 인코딩

In [21]:
tokenizer = Tokenizer()

In [22]:
tokenizer.fit_on_texts(df["token"])

In [23]:
tokenizer.word_index

{'하다': 1,
 '의': 2,
 '에': 3,
 '을': 4,
 '를': 5,
 '이': 6,
 '있다': 7,
 '은': 8,
 '되다': 9,
 '및': 10,
 '는': 11,
 '으로': 12,
 '등': 13,
 '한': 14,
 '움': 15,
 '돋다': 16,
 '가': 17,
 '경우': 18,
 '수': 19,
 '사업': 20,
 '과': 21,
 '에서': 22,
 '로': 23,
 '시설': 24,
 '시': 25,
 '자': 26,
 '인': 27,
 '신청': 28,
 '제': 29,
 '관리': 30,
 '지원': 31,
 '주택': 32,
 '일': 33,
 '이다': 34,
 '년': 35,
 '적': 36,
 '센터': 37,
 '또는': 38,
 '하고': 39,
 '서울시': 40,
 '받다': 41,
 '계획': 42,
 '원': 43,
 '운영': 44,
 '이상': 45,
 '월': 46,
 '와': 47,
 '교육': 48,
 '사항': 49,
 '어떻다': 50,
 '업무': 51,
 '내용': 52,
 '서': 53,
 '서울': 54,
 '지': 55,
 '공': 56,
 '중': 57,
 '것': 58,
 '관련': 59,
 '대상': 60,
 '이용': 61,
 '동': 62,
 '지역': 63,
 '위': 64,
 '무엇': 65,
 '관': 66,
 '따르다': 67,
 '후': 68,
 '기준': 69,
 '도시': 70,
 '에는': 71,
 '건축물': 72,
 '설치': 73,
 '확인': 74,
 '신고': 75,
 '가능하다': 76,
 '해당': 77,
 '방법': 78,
 '내': 79,
 '되어다': 80,
 '공무원': 81,
 '없다': 82,
 '제조': 83,
 '규정': 84,
 '변경': 85,
 '홈페이지': 86,
 '의하다': 87,
 '건축': 88,
 '주민': 89,
 '법': 90,
 '정보': 91,
 '구': 92,
 '기관': 93,
 '도로': 94,
 

In [24]:
tokenizer.word_counts

OrderedDict([('아빠', 16),
             ('육아휴직', 10),
             ('장려', 6),
             ('금', 89),
             ('업무', 588),
             ('개요', 171),
             ('남성', 3),
             ('근로자', 41),
             ('의', 5030),
             ('을', 4237),
             ('하고', 693),
             ('양육', 15),
             ('에', 4578),
             ('따르다', 484),
             ('경제', 78),
             ('적', 729),
             ('부담', 117),
             ('완화', 17),
             ('함', 353),
             ('으로써', 43),
             ('일과', 7),
             ('가정', 104),
             ('생활', 208),
             ('양립', 2),
             ('및', 1880),
             ('가족', 151),
             ('친', 10),
             ('화', 105),
             ('인', 863),
             ('사회', 253),
             ('환경', 276),
             ('조성', 88),
             ('지원', 772),
             ('대상', 531),
             ('신청', 839),
             ('일', 750),
             ('기준', 479),
             ('년', 732),
             ('이상', 618),
       

In [25]:
total_cnt = len(tokenizer.word_index)
total_cnt

11519

In [26]:
# 사용 단어 수를 지정하여 토큰화 : num_words = 1000
num_words = 1000
tokenizer = Tokenizer(num_words = num_words)
tokenizer.fit_on_texts(df["token"])

In [27]:
df["token"] = tokenizer.texts_to_sequences(df["token"])

In [28]:
df.head()

,분류,문서,token
0,복지,아빠 육아휴직 장려금아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려...,"[422, 422, 51, 218, 838, 2, 4, 39, 3, 67, 482,..."
1,경제,서울산업진흥원 서울메이드란서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새...,"[53, 283, 337, 658, 43, 53, 283, 175, 54, 337,..."
2,복지,광진맘택시 운영임산부영아 양육가정 전용 택시광진맘택시 운영임산부영아 양육가정 전용 ...,"[734, 973, 44, 358, 493, 973, 734, 973, 44, 35..."
3,복지,마포 뇌병변장애인 비전센터마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 ...,"[975, 196, 590, 37, 975, 196, 590, 37, 975, 19..."
4,행정,년도 중고 신입생 입학준비금 지원년도 중고 신입생 입학준비금 지원 업무개요 서울...,"[602, 675, 422, 31, 602, 675, 422, 31, 51, 218..."


### 길이 확인

In [29]:
length = [len(i) for i in df["token"].tolist()]

In [30]:
# 길이 평균값과 중간값
print(np.mean(length), np.median(length), np.min(length), np.max(length))

84.84043051006083 55.0 2 2913


### 데이터 분할

In [31]:
x = df["token"]
y = df["분류"]

In [32]:
labels = {0 : "행정", 1 : "경제", 2 : "복지"}
labels_rev = {"행정" : 0, "경제" : 1, "복지" : 2}

In [33]:
y = y.map(labels_rev)

In [34]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y, random_state = 7)

In [35]:
x_sub, x_val, y_sub, y_val = train_test_split(x_train, y_train, test_size = 0.2, stratify = y_train, random_state = 7)

In [36]:
maxlen = 60

sub_seq = pad_sequences(x_sub, maxlen = maxlen, truncating = "pre")
val_seq = pad_sequences(x_val, maxlen = maxlen, truncating = "pre")
test_seq = pad_sequences(x_test, maxlen = maxlen, truncating = "pre")

In [37]:
y_oh_sub = keras.utils.to_categorical(y_sub)
y_oh_val = keras.utils.to_categorical(y_val)
y_oh_test = keras.utils.to_categorical(y_test)

### 모델 구성 및 훈련

In [38]:
model = keras.Sequential([
    keras.layers.Embedding(num_words, 128, input_shape = (maxlen,)),
    keras.layers.Dropout(0.3),
    keras.layers.Conv1D(128, 5, activation = "relu"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool1D(4),
    keras.layers.LSTM(64, dropout = 0.5, return_sequences = True),
    keras.layers.LSTM(64, dropout = 0.5),
    keras.layers.Dense(3, activation = "softmax")
])

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [39]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 60, 128)             │         128,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 60, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 56, 128)             │          82,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 56, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 14, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 14, 64)              │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 293,187 (1.12 MB)

 Trainable params: 292,931 (1.12 MB)

 Non-trainable params: 256 (1.00 KB)

In [40]:
rmsprop = keras.optimizers.RMSprop(learning_rate = 5e-4)
model.compile(loss = "categorical_crossentropy", optimizer = rmsprop, metrics = ["accuracy"])
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 4, restore_best_weights = True)

In [41]:
history = model.fit(sub_seq, y_oh_sub, batch_size = 16, epochs = 100,
                   validation_data = (val_seq, y_oh_val),
                   callbacks = [early_stopping_cb])

Epoch 1/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.4927 - loss: 0.9763 - val_accuracy: 0.6082 - val_loss: 0.9715
Epoch 2/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5870 - loss: 0.8598 - val_accuracy: 0.6959 - val_loss: 0.8895
Epoch 3/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7398 - loss: 0.6686 - val_accuracy: 0.7602 - val_loss: 0.7347
Epoch 4/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7905 - loss: 0.5309 - val_accuracy: 0.7398 - val_loss: 0.6181
Epoch 5/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8618 - loss: 0.3883 - val_accuracy: 0.7544 - val_loss: 0.5892
Epoch 6/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8753 - loss: 0.3483 - val_accuracy: 0.7632 - val_loss: 0.6294
Epoch 7/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9032 - loss: 0.2840 - val_accuracy: 0.7515 - val_loss: 0.7510
Epoch 8/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9122 - loss: 0.2250 - val_accuracy: 0.

In [59]:
model.evaluate(test_seq, y_oh_test)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7888 - loss: 0.5749


[0.6088308691978455, 0.7686915993690491]

### 정확도 구하기

In [60]:
pred = model.predict(test_seq)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [65]:
accuracy = sum([int(np.argmax(pred[i]) == y_test.iloc[i]) for i in range(len(pred))]) / len(pred)

In [66]:
# 정확도 출력
print(accuracy)

0.7686915887850467
